In [5]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(7, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 7),
            nn.Sigmoid()  # Применяем сигмоид для ограничения значений в диапазоне [0, 1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

data = pd.read_csv("Seed_Data.csv", header=None)
X = data.iloc[:, :-1].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Конвертация данных в тензоры PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

model = Autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

num_epochs = 190
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, X_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Применение модели для восстановления данных
model.eval()
with torch.no_grad():
    decoded_data = model(X_tensor)

# Восстановление данных и их обратное масштабирование
decoded_data = decoded_data.numpy()
decoded_data = scaler.inverse_transform(decoded_data)


mse = mean_squared_error(X, decoded_data)
# Вычисление средней абсолютной ошибки
mae = mean_absolute_error(X, decoded_data)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

Epoch [10/190], Loss: 1.2699
Epoch [20/190], Loss: 1.2489
Epoch [30/190], Loss: 1.2245
Epoch [40/190], Loss: 1.1948
Epoch [50/190], Loss: 1.1590
Epoch [60/190], Loss: 1.1175
Epoch [70/190], Loss: 1.0723
Epoch [80/190], Loss: 1.0224
Epoch [90/190], Loss: 0.9682
Epoch [100/190], Loss: 0.9140
Epoch [110/190], Loss: 0.8638
Epoch [120/190], Loss: 0.8224
Epoch [130/190], Loss: 0.7906
Epoch [140/190], Loss: 0.7660
Epoch [150/190], Loss: 0.7465
Epoch [160/190], Loss: 0.7305
Epoch [170/190], Loss: 0.7173
Epoch [180/190], Loss: 0.7063
Epoch [190/190], Loss: 0.6970
Mean Squared Error (MSE): 1.225581275459998
Mean Absolute Error (MAE): 0.6857784444788848
